In [86]:
!pip3 install py_vollib 

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy.stats as stats
from scipy.stats import shapiro
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import warnings
import matplotlib as mpl
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

#### Underlying Asset: S&P 500

In [2]:
asset = yf.download("^GSPC", start= "2016-06-17", end="2021-06-17")
asset = pd.DataFrame(asset["Close"].dropna())
asset1 = yf.download("^GSPC", start= "2020-07-03", end="2021-07-03")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# 1 Time - series analysis

### 1.1 Log-normal properity of GBM
We know that stock prices follow a Geometric Brownian Motion. One of the properties of GBM is that stock prices are log-normal.
Hence, logarithm of stock prices are normally distributed.In what follows we will try to show that if log returns of the stock prices have normal distribution. We do this for different time lengths - 1, 2 and 5 years

We now calculate log-returns, $\mu, \sigma$ and check for normality

In [ ]:
asset_returns = np.log(asset/asset.shift(1))
asset_returns = asset_returns.dropna()

Calculating $\mu, \sigma$ for 1, 2 and 5 years

In [ ]:
daily_mean_1 = asset_returns["2020-06-17":"2021-06-17"].mean()
daily_mean_2 = asset_returns["2019-06-17":"2021-06-17"].mean()
daily_mean_5 = asset_returns["2016-06-17":"2021-06-17"].mean()

In [ ]:
daily_std_1 = asset_returns["2020-06-17":"2021-06-17"].std()
daily_std_2 = asset_returns["2019-06-17":"2021-06-17"].std()
daily_std_5 = asset_returns["2016-06-17":"2021-06-17"].std()

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (18,5))

x1 = np.arange(-0.24,0.2, 0.001)
x2 = np.arange(-0.22,0.2, 0.001)
x3 = np.arange(-0.2,0.2, 0.001)

ax[0].hist(asset_returns["2020-06-17":"2021-06-17"], bins = 100)
ax[0].plot(x1, stats.norm.pdf(x1, daily_mean_1, daily_std_1))
ax[0].set_title("1 Year Returns")

ax[1].hist(asset_returns["2019-06-17":"2021-06-17"], bins = 100)
ax[1].plot(x2, stats.norm.pdf(x2, daily_mean_2, daily_std_2))
ax[1].set_title("2 Year Returns")

ax[2].hist(asset_returns["2016-06-17":"2021-06-17"], bins = 100)
ax[2].plot(x3, stats.norm.pdf(x3, daily_mean_5, daily_std_5))
ax[2].set_title("5 Year Returns");

From the histograms we can see that returns for 1, 2 and 5 years seem to be not normally distributed, but, in order to prove normality we do a more rigorous test - Shapiro - Wilk test

The Shapiro-Wilk test evaluates a data sample and quantifies how likely it is that the data was drawn from a Gaussian distribution

- p < 0.05, reject H0 - not normal

- p > 0.05, fail to reject H0 - normal


In [ ]:
stat, p = shapiro(asset_returns["2020-06-17":"2021-06-17"])
print(f"p value for 1 year returns: {p}")
stat, p = shapiro(asset_returns["2019-06-17":"2021-06-17"])
print(f"p value for 2 year returns: {p}")
stat, p = shapiro(asset_returns["2016-06-17":"2021-06-17"])
print(f"p value for 5 year returns: {p}")

p values for all all the time lengths are less than 0.05. Hence, we reject the H0 and confirm that our data sets are not normaly distributed.

### 1.2 Markov properity of GBM
A Markov process is a particular type of stochastic process where only the current value of a variable is relevant for predicting the future. So, in our case past history of our variable should be irrelevat to predict the future values.We should be able to use only present data to make a prediction about tomorrow's price. We can use plot_acf and plot_pacf functions to see the relationship between todays stock rices and past prices. 

In [ ]:
fig, axes = plt.subplots(3,2, figsize = (16,12))
plot_acf(asset["2020-06-17":"2021-06-17"], ax = axes[0,0]) 
axes[0,0].set_title("Autocorrelation for 1-year stock prices")

plot_acf(asset["2019-06-17":"2021-06-17"], ax = axes[1,0])   
axes[1,0].set_title("Autocorrelation for 2-year stock prices")

plot_acf(asset["2016-06-17":"2021-06-17"], ax = axes[2,0])
axes[2,0].set_title("Autocorrelation for 5-year stock prices")



plot_pacf(asset["2020-06-17":"2021-06-17"], ax = axes[0,1])   
axes[0,1].set_title("Partial Autocorrelation for 1-year stock prices")

plot_pacf(asset["2019-06-17":"2021-06-17"], ax = axes[1,1])  
axes[1,1].set_title("Partial Autocorrelation for 2-year stock prices")

plot_pacf(asset["2016-06-17":"2021-06-17"], ax = axes[2,1])
axes[2,1].set_title("Partial Autocorrelation for 5-year stock prices");

- plot_acf function gives us autocorrolation - correlation of a given time series with its lagged version of itself. Here we have direct and indirect effects of lagged versions
- plot_pacf function gives us partial autocorrelation - at lag k, this is the correlation between series values that are k intervals apart. In partial autocorrelation indirect effects have been removed

We can see that there is a strong autocorrelation of our time series with its lagged versions, but, this correlation is getting smaller as the number of laggs increase.

The main finding comes from partial autocorrelation function. It is clear that only present value of the stock price is relevat to make a prediction about tommorow's price. If we consider the first lag - St - as the future value and the second lag - S0 - as present value. It is only S0 that we can use as a predictor for St. All the remaining past prices -lags - have statistically insignificant correlation with St. This backs up Markov process - only the current value of a variable is relevant for predicting the future.

### 1.3 Continuous sample path properity of GBM
Sample path of GBM is continuous as a function of time and jagged. A continuous function is a function that does not have any abrupt changes in value  and there are no breaking points - discontinuities. A GBM function is also very jagged. No matter how much we zoom in, we can still observe jagged patterns. Such a function is also non-differentiable. Next we plot our asset to see if it has these features.

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (18,5))
ax[0].plot(asset["2020-06-17":"2021-06-17"])
ax[0].set_title("1 Year Prices")
ax[0].set_ylabel("Price")
ax[0].set_xlabel("Time")

ax[1].plot(asset["2019-06-17":"2021-06-17"])
ax[1].set_title("2 Year Prices")
ax[1].set_ylabel("Price")
ax[1].set_xlabel("Time")

ax[2].plot(asset["2016-06-17":"2021-06-17"])
ax[2].set_title("5 Year Prices")
ax[2].set_ylabel("Price")
ax[2].set_xlabel("Time");

In fact, we can clearly see that sample path of our asset is extremely jagged and continuous.

We conclude that even though our underlying asset returns are not normally distributed (however they resemble normal distribution symetry), it is in fact a Markov process and its sample path is a continuous function of time and jagged. Based on what we observe above we assume that our underlying asset follows a geometric Brownian Motion and proceed to the second part - finding implyed volatility with Black-Scholes-Merton formula.

------

#  2 Finding Implied Volatility
Implied volatility is calculated by taking the market price of the option, entering it into the Black-Scholes formula, and back-solving for the value of the volatility

### 2.1 Black-Scholes-Merton formula

First we have to implement Black-Scholes formula:
\begin{equation}C = N(d_{1})Sd_{t} - N(d_{2})Ke^{-rt}\end{equation}

\begin{equation}d_{1} = \frac{ln(\frac{S_{t}}{K})+(r+\frac{\sigma^2}{2})}{\sigma\sqrt{t}} \end{equation}

\begin{equation}d_{2} = d_{1} - \sigma\sqrt{t} \end{equation}

- S0: value of underlying
- K: strike price
- T: time to maturity (year fraction)
- r: risk free rate
- sigma_est: estimated value for sigma - to initiate the process
- sigma: volatility factor (historical volatility)
- C0: market value of option

In [18]:
def call_value(S0, K, T, r, sigma):
    S0 = float(S0)
    d1 = (math.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = (math.log(S0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    value = (S0 * stats.norm.cdf(d1, 0.0, 1.0) - K * math.exp(-r * T) * stats.norm.cdf(d2, 0.0, 1.0))
    return value

- call_value function implements Black-Scholes-Merton formula and finds the value of European call option

In [19]:
def vega(S0, K, T, r, sigma):
    S0 = float(S0)
    d1 = (math.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    vega = S0 * stats.norm.pdf(d1, 0.0, 1.0) * math.sqrt(T)
    return vega

- Vega is the Greek that measures an option’s sensitivity to implied volatility
- It is the change in the option’s price for a one-point change in implied volatility

Formula for implied vaolatility:

\begin{equation}\sigma_{iv} = \frac{C-C_{0}}{V} \end{equation}

- C: Calculated value, using BSM formula
- C0: Market value
- V: Vega

In [79]:
def imp_vol(S0, K, T, r, C0, sigma_estimate, it=400):
    for i in range(it):
        sigma_estimate -= ((call_value(S0, K, T, r, sigma_estimate) - C0) / vega(S0, K, T, r, sigma_estimate))
        return sigma_estimate


- What we are doing here is basically giving call_value function our estimated volatility - sigma_estimate and other parameters to get the value of option. Because we have the real value of the option - C0 we can see if sigma_estimate is close enough to the real implied volatility which would give us the real value of the option(C0). We find value of the option with call_value and deduct C0 from it and divide the result by vega. We do this process 100 times - trail and error. When the value from this eqaution is 0, this will be the real implied volatility - at least something very close to it.

### 2.1 Applying BSM formula to find implied volatililty

Now we use our functions to find the implied volatility

- Underlying: S&P 500
- Option:SPX
- Symbol:SPXW210806C04340000 
- Strike:4340
- Price: 59.12
- Expiration date: Fri Aug 06 2021

- Because our option's expiration date is one month away, we use 1 month US treasury rate as risk free rate which is 0.048%

- Data has been taken from https://www.cboe.com/

- This data has been taken 2 July, 2021 and is subject to change. This would mean that implied volatility found here might be different than the implied volatility calculated at a different future time and to get the right iplied volatility parameters should be updated

In [80]:
S0 = 15    
K = 13
T = 0.25
r = 0.05
sigma = 0
sigma_estimate = 0.4
C0 = 2.5

In [81]:
imp_vol = imp_vol(S0, K, T, r, C0, sigma_estimate, it=100)

In [82]:
print(f"Model implied volatility: {(100*imp_vol).round(4)}%")
print(f"Real implied volatility: 11%")

Model implied volatility: 39.6445%
Real implied volatility: 11%
